In [1]:
from pdf_page_annotator import PDFAnnotator

annotator = PDFAnnotator('doefund.pdf', verbose=True)

Initialized PDFAnnotator with doefund.pdf


In [2]:
annotator.run_extraction_pipeline()

Page 0 ...
Page 1 ...
Page 2 ...
Page 3 ...
Page 4 ...
Page 5 ...
Page 6 ...
Finished annotating table of contents
Extracting contents ...
Extracting contents from page 2 ...
Extracting contents from page 3 ...
Extracting contents from page 4 ...
Extracting contents from page 5 ...
Extracted contents
Extracting contents from chunk ...
Extracting contents from chunk ...
Extracting contents from chunk ...
Extracting contents from chunk ...
Extracting contents from chunk ...
Extracting contents from chunk ...
Extracting contents from chunk ...
Extracting contents from chunk ...
Extracting contents from chunk ...
Extracting contents from chunk ...
Extracting contents from chunk ...
Extracting contents from chunk ...
Extracting contents from chunk ...
Extracting contents from chunk ...
Extracting contents from chunk ...
Extracting contents from chunk ...
Extracting contents from chunk ...
Extracting contents from chunk ...
Extracting contents from chunk ...


In [3]:
content = annotator.contents
content[0].unique_title, content[0].start_page, content[0].end_page

('I. Funding Opportunity Description', 1, 12)

In [5]:
# extract all the tuples of (start_page, end_page) for each unique_title
heading_tuples = []
for c in content:
    heading_tuples.append((c.unique_title, c.start_page, c.end_page))
len(heading_tuples)

126

In [6]:
# export the tuples into a csv file names toc.csv
import csv
with open('toc.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(heading_tuples)

In [16]:
PROMPT = """Given are the Table of Contents of an RFP document. Based on the user query, return the section and it's contents where the requested information is found.
Each table of content tuple contains the unique heading, start page and end page of the section.
Table of Contents:
{toc}
Query:
{query}
---
Return a list of tuples as expressed above.
Results:"""
print(PROMPT.format(toc=heading_tuples, query="Where can I find all the evaluation criteria?"))

Given are the Table of Contents of an RFP document. Based on the user query, return the section and it's contents where the requested information is found.
Each table of content tuple contains the unique heading, start page and end page of the section.
Table of Contents:
[('I. Funding Opportunity Description', 1, 12), ('A. Background and Context', 1, 2), ('i. Program Purpose', 2, 2), ('ii. Technology Space and Strategic Goals', 5, 5), ('B. Topic Areas', 6, 6), ('C. Applications Specifically Not of Interest', 11, 11), ('D. R&D Community Benefits Plan (CBP)', 12, 12), ('E. Authorizing Statutes', 12, 12), ('F. Notice of Bipartisan Infrastructure Law– Specific Requirements', 13, 13), ('II. Award Information', 14, 15), ('A. Award Overview', 14, 14), ('B. DOE Funding Agreements', 14, 15), ('III. Eligibility Information', 15, 15), ('A. Eligible Applicants', 15, 16), ('B. Cost Sharing', 16, 19), ('C. Compliance Criteria', 20, 20), ('D. Responsiveness Criteria', 21, 21), ('E. Other Eligibility 

In [23]:
from llama_index.llms.openai import OpenAI
llm = OpenAI(model="gpt-3.5-turbo")
response = llm.complete(PROMPT.format(toc=heading_tuples, query="Where can I find all the evaluation criteria? Return the section start and end item.")).text

In [24]:
response

"[('V. Application Review Information', 56, 61), ('A. Technical Review Criteria', 56, 61), ('i. Concept Papers', 56, 61), ('ii. Full Applications', 57, 61), ('B. Standards for Application Evaluation', 61, 61), ('C. Other Selection Factors', 61, 61), ('D. Evaluation and Selection Process', 62, 63), ('E. Anticipated Notice of Selection and Award Negotiation Dates', 63, 63)]"

In [25]:
import ast
response = ast.literal_eval(response)
response

[('V. Application Review Information', 56, 61),
 ('A. Technical Review Criteria', 56, 61),
 ('i. Concept Papers', 56, 61),
 ('ii. Full Applications', 57, 61),
 ('B. Standards for Application Evaluation', 61, 61),
 ('C. Other Selection Factors', 61, 61),
 ('D. Evaluation and Selection Process', 62, 63),
 ('E. Anticipated Notice of Selection and Award Negotiation Dates', 63, 63)]

In [26]:
page_start = response[0][1]
page_end = response[-1][2]
page_start, page_end

(56, 63)

In [31]:
from pypdf import PdfReader
reader = PdfReader('doefund.pdf')
text = reader.pages[page_start+5:page_end+5]
text = [t.extract_text() for t in text]
text

[' \nQuestions about this FOA? Email : DE-FOA-0003120@netl.doe.gov  \nProblems with EERE eXCHANGE ? Email EERE -eXCHANGE Support@hq.doe.gov  Include FOA name and number in \nsubject line.  \n 56 \n \nxiv. Foreign Collaboration Considerations  \na. Consideration of new collaborations with foreign entities , organizations,  and \ngovernments . The re cipient will be required to provide DOE with advanced \nwritten notification of any potential collaboration with foreign entities , \norganizations,  or governments in connection with its DOE -funded award scope. \nThe recipient will then be required to await further guidance from DOE prior to \ncontacting the proposed foreign entity , organizations,  or government regarding \nthe potential collaboration or negotiating the terms of any potential agreement.  \n \nb. Existing collaborations with foreign entitie s, organizations,  and governments . \nThe recipient will be required to provide DOE with a written list of all existing foreign colla

In [41]:
_EXTRACTION_PROMPT = """For the given excerpt from an RFP document, it is suspected that it contains the evaluation criteria and hard requirements that the application will be evaluated against.
As an advanced language model, your job is to remove the fluff and return these criteria and requirements.
Output a list of these criteria and requirements with each item separated by a newline.
Ensure that the items are hard conditionals and not soft language.
Excerpt:
{excerpt}
---
Start each line with a hyphen and a space. Start each item with something like, "The applicant must" or "The application should" or something similar.
Results:"""
EXTRACTION_PROMPT = """For the given excerpt from an RFP document, it is suspected that it contains the evaluation criteria and hard requirements that the application will be evaluated against.
As an advanced language model, your job is to remove the fluff and return these criteria and requirements.
Output a list of these criteria and requirements with each item separated by a newline.
Ensure that the items are hard conditionals and not soft language.
Excerpt:
{excerpt}
---
Start each line with a hyphen and a space. Each item should have a cause and effect format like, "The application will be evaluated on ... so the applicant must ..." or "The application should ... so the applicant must ..."
Results:"""
print(EXTRACTION_PROMPT.format(excerpt=text[0]))

For the given excerpt from an RFP document, it is suspected that it contains the evaluation criteria and hard requirements that the application will be evaluated against.
As an advanced language model, your job is to remove the fluff and return these criteria and requirements.
Output a list of these criteria and requirements with each item separated by a newline.
Ensure that the items are hard conditionals and not soft language.
Excerpt:
 
Questions about this FOA? Email : DE-FOA-0003120@netl.doe.gov  
Problems with EERE eXCHANGE ? Email EERE -eXCHANGE Support@hq.doe.gov  Include FOA name and number in 
subject line.  
 56 
 
xiv. Foreign Collaboration Considerations  
a. Consideration of new collaborations with foreign entities , organizations,  and 
governments . The re cipient will be required to provide DOE with advanced 
written notification of any potential collaboration with foreign entities , 
organizations,  or governments in connection with its DOE -funded award scope. 
The r

In [42]:
llm.model

'gpt-3.5-turbo'

In [43]:
_conditions = []
for t in text:
    response = llm.complete(EXTRACTION_PROMPT.format(excerpt=t)).text
    _conditions.append(response)

In [44]:
# export the conditions into a txt file
with open('conditions.txt', 'w') as file:
    file.write('\n'.join(_conditions))